In [ ]:
## Train Keras CNN Model on AWS Sagemaker

In [ ]:
import os
import sys

import sagemaker
from sagemaker.tensorflow import TensorFlow
import tensorflow as tf
from tensorflow import keras
import boto3
from botocore.exceptions import ClientError

In [ ]:
SM_SESSION = sagemaker.Session()
SM_ROLE = sagemaker.get_execution_role()

TOPDIR = os.getcwd()

# Our SM jobs will use this bucket
SM_WORKING_S3_BUCKET = "dev-eadatasciencesb-us-east-1-sagemaker-pdamore"

EPOCHS = 20
BATCH_SIZE = 256
LEARNING_RATE = 0.01

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

np.savez('./data/training', image=train_images, label=train_labels)
np.savez('./data/validation', image=test_images, label=test_labels)

In [ ]:
# Upload the file
def upload_file(file_name, bucket, object_name=None):
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

upload_file('data/training.npz', SM_WORKING_S3_BUCKET, 's3://'+SM_WORKING_S3_BUCKET+'/training')
upload_file('data/validation.npz', SM_WORKING_S3_BUCKET, 's3://'+SM_WORKING_S3_BUCKET+'/validation')

In [ ]:
model_dir = '/opt/ml/model'

train_instance_type = 'ml.p2.xlarge'
#train_instance_type = 'local'

# Path is relative to the src_dir
hyperparameters = {'epochs' : EPOCHS,
                   'batch-size' : BATCH_SIZE,
                   'learning-rate' : LEARNING_RATE
                  }

inputs = {'training': training_input_path, 'validation': validation_input_path}
print(inputs)

In [ ]:
tf_estimator = TensorFlow(entry_point='mnist_keras_tf.py', 
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='ml.p3.2xlarge',
                          framework_version='1.12',
                          py_version='py3',
                          script_mode=True,
                          hyperparameters=hyperparameters
                         )

In [ ]:
tf_estimator.fit(inputs)

In [ ]:
## Deploy

In [ ]:
import time

tf_endpoint_name = 'tf-fmnist-'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

#tf_predictor = tf_estimator.deploy(initial_instance_count=1,
#                                   instance_type='ml.p2.xlarge')      # $1.361/hour in eu-west-1

tf_predictor = tf_estimator.deploy(initial_instance_count=1,
                         instance_type='ml.c5.large',
                         endpoint_name=tf_endpoint_name) 

In [ ]:
 %matplotlib inline
import random
import matplotlib.pyplot as pyplot

num_samples = 5
indices = random.sample(range(train_images.shape[0] - 1), num_samples)
images = train_images[indices]/255
labels = train_labels[indices]

for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(images[i].reshape(28, 28), cmap='gray')
    plt.title(labels[i])
    plt.axis('off')
    
prediction = tf_predictor.predict(images.reshape(num_samples, 28, 28, 1))['predictions']
prediction = np.array(prediction)
predicted_label = prediction.argmax(axis=1)
print('Predicted labels are: {}'.format(predicted_label))

In [ ]:
SM_SESSION.delete_endpoint(endpoint_name=tf_endpoint_name)